**Getting the Data from Site**

In [27]:
import urllib3

# Create a PoolManager instance
http = urllib3.PoolManager()

# Specify the URL you want to request
url = "https://www.gutenberg.org/cache/epub/11/pg11-images.html" #link to the Book
url_2 = "https://www.gutenberg.org/cache/epub/11/pg11.txt" #Link of plain text of the above mentioned book

# Send an HTTP GET request
Data = http.request("GET", url_2)

# Read and print the response content
Data = Data.data.decode('utf-8')


In [28]:
import numpy as np

# load ascii text and covert to lowercase
raw_text = Data
raw_text = raw_text.lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  167711
Total Vocab:  66
